In [85]:
import os
import subprocess
import numpy as np
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
from photutils import CircularAperture
from photutils import aperture_photometry
from astropy import wcs
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import requests
import time
from bs4 import BeautifulSoup

In [2]:
# diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
# medianDir = '/media/mj1e16/PP AV-TV/medians/'
# imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'

In [3]:
diffDir = '/data/mj1e16/kepler/properDiff/'
medianDir = '/data/mj1e16/kepler/medians/'
imageDir = '/data/mj1e16/kepler/cal/'

In [4]:
diffImages = os.listdir(diffDir)
diffImages = [x for x in diffImages if '.fits' in x]

medianImages = os.listdir(medianDir)
medianImages = [x for x in medianImages if '.fits' in x]

images = os.listdir(imageDir)
images = [x for x in images if '.fits' in x]

In [221]:
#unmatchedObjects = ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')
unmatchedObjects =  ascii.read('/home/mj1e16/keplerPhotometry/sexdiffOutNew2.csv')

In [68]:
#firstunmatchedObjects = ascii.read('/home/mj1e16/outTablesImproved/firsthalf.csv')
secondunmatchedObjects = ascii.read('/home/mj1e16/outTablesImproved/secondhalf.csv')

In [69]:
unmatchedObjects = secondunmatchedObjects

In [70]:
indexlist = np.linspace(0,len(unmatchedObjects)-1,len(unmatchedObjects))
intexlist = [int(x) for x in indexlist]
print(len(intexlist))
print(len(unmatchedObjects))
print(int(intexlist[-1]))

286773
286773
286772


In [71]:
xpos = np.array(unmatchedObjects['X_IMAGE'])
ypos = np.array(unmatchedObjects['Y_IMAGE'])
imname = np.array(unmatchedObjects['IMAGE_NAME'])
unmatchedObjects['index'] = intexlist
#indexlist = np.linspace(0,len(unmatchedObjects),len(unmatchedObjects)+1)
allnames = np.unique(imname)

In [72]:
uni = np.unique(imname)
print(len(uni))

2166


In [12]:
def load_wcs_from_file(filename,pixCoord,ext=0):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)
    print(filename)
    # Parse the WCS keywords in the primary HDU
    print(int(ext))
    w = wcs.WCS(hdulist[int(ext)].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [13]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='5'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [14]:
def appendToDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='5'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = ''
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'a') as f:
            f.write(bigString)

In [15]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [16]:
def findMatchesNoCondition(image,diffname,tab,match,df,bigdf,max_sep = 2.5 * u.arcsec,magsurplus=-3):
    
    
    number1 = match['NUMBER'] # original image
    ra1 = match['RA']
    dec1 = match['DEC']

    number2 = tab['index'] # difference image
    ra2 = tab['RA']
    dec2 = tab['DEC']    
    
    makeDS9RegFile([tab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='red')

    cat1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat2.match_to_catalog_sky(cat1)
    sep_constraint = d2d < max_sep
    objectsFound = number2[sep_constraint]
    matchdf = df[df['index'].isin(objectsFound)]
    matchdf.sort_values(['Y_IMAGE'])
    ognumber = matchdf['index'].tolist()
    ogflux = matchdf['FLUX_BEST'].tolist()
    matchTab = Table.from_pandas(matchdf)

    appendToDS9RegFile([matchTab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='blue',radius='10')
    
    idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)
    sep_constraint = d2d < max_sep
    #istrue = [x for x in sep_constraint if x == True]
    objectsFound = number1[sep_constraint]
    origdf = bigdf[bigdf['NUMBER'].isin(objectsFound)]
    origdf.sort_values(['Y_IMAGE'])
    origflux = origdf['FLUX_BEST'].tolist()
    origMag = origdf['MAG_BEST'].tolist()
    
    fluxdiff = [(x-y) for x,y in zip(origflux,ogflux)]
    magdiff = [-2.5*np.log10(x/y) for x,y in zip(origflux,fluxdiff)]
    #print(magdiff)
    goodNums = [x for x,y in zip(ognumber,magdiff) if y < magsurplus] # fine for now, in future change to not rely on the 
    magdiffColumn = [x for x in magdiff if x < magsurplus]            # Y_iMAGE position of the objects
    magorigColumn = [x for x,y in zip(origMag,magdiff) if y < magsurplus]
    finaldf = matchdf[matchdf['index'].isin(goodNums)]
    finaldf['magDiff'] = magdiffColumn
    finaldf['MAG_BEST_ORIG'] = magorigColumn
    if len(finaldf) != 0:
        #print(finaldf)
        finalTab = Table.from_pandas(finaldf)
        appendToDS9RegFile([finalTab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='green',radius='15')
    return finaldf

In [17]:
def showImage(images,regfile):
    commands = ['ds9']
    for x in range(len(images)):
        commands.extend([images[x],'-regions',regfile,'-scale','mode','zscale'])
    commands.extend(['-frame','lock','image'])
    #print(commands)
    subprocess.call(commands)

In [222]:
def openFromTable(table,ind,imDir=imageDir,diffDir=diffDir,medianDir=medianDir):
    ra = table['RA'][ind]
    dec = table['DEC'][ind]
    xim = table['X_IMAGE'][ind]
    yim = table['Y_IMAGE'][ind]
    image = table['IMAGE_NAME'][ind]
    median = table['MEDIAN_NAME'][ind]
    diff = table['DIFF_NAME'][ind]
    with open('/home/mj1e16/outTablesImproved/testregImage_{}.reg'.format(ind),'w') as f:
        f.write('global color=red\nimage\ncircle({0},{1},{2})\ncircle({0},{1},{3})'.format(xim,yim,5.,60.))
    with open('/home/mj1e16/outTablesImproved/testregWCS_{}.reg'.format(ind),'w') as f:
        f.write('global color=red\nICRS\ncircle({0},{1},{2})\ncircle({0},{1},{3})'.format(ra,dec,3./3600,24./3600.))
    
    command = ['ds9']
    images= [imDir+image,diffDir+diff,medianDir+median]
    for x in range(3):
        command.extend([images[x],'-regions','/home/mj1e16/outTablesImproved/testreg_{}.reg'.format(ind),'-scale','mode','zscale'])
    getPanStarCutOut(ra,dec,size=240)
    panCommand = ['/home/mj1e16/cutOuts/cutout.fits','-regions','/home/mj1e16/outTablesImproved/testregWCS_{}.reg'.format(ind)]
    command.extend(panCommand)
    command.extend(['-frame','lock','wcs'])
    print(ra,dec,image)
    subprocess.call(command)

In [219]:
def getPanStarCutOut(ra,dec,size=240,outname='cutout.fits'):
    url = 'https://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={}%2C{}&filter=color&filter=g&filter=r&filter=i&filter=z&filter=y&filetypes=stack&auxiliary=data&size={}&output_size=0&verbose=0&autoscale=99.500000&catlist='.format(ra,dec,size)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    th = soup.findAll('td')
    try:
        link = th[1]
        downLink = link.findAll('img')
        downCut = downLink[0]['src']
        loc1 = downCut.find('cgi-bin')
        loc2 = downCut.find('=/') + len('=/')
        if loc1 != -1:
            if loc2 != -1:
                newCut = downCut[:loc1] + downCut[loc2:]

        loc3 = newCut.find('&')
        smallCut = newCut[:loc3]
        http = 'https:'+smallCut
        print(http)
        subprocess.call(['wget','-O','/home/mj1e16/cutOuts/{}'.format(outname),http])
    except:
        print('WARNING - panstarrs image note found ra = {}, dec = {}'.format(ra,dec))


In [ ]:
for x in range(len(matches)):
    openFromTable(matches,x)

https://ps1images.stsci.edu/rings.v3.skycell/2275/094/rings.v3.skycell.2275.094.stk.g.unconv.fits
(281.3900633996704, 47.862257782235325, 'kplr2009115053616_ffi-cal.fits[15]')


In [183]:
a = getPanStarCutOut(0,0)

<img height="240" src="//ps1images.stsci.edu/cgi-bin/fitscut.cgi?red=/rings.v3.skycell/1232/094/rings.v3.skycell.1232.094.stk.g.unconv.fits&amp;x=0.000000&amp;y=0.000000&amp;size=240&amp;wcs=1&amp;asinh=True&amp;autoscale=99.500000" width="240"/>
//ps1images.stsci.edu/cgi-bin/fitscut.cgi?red=/rings.v3.skycell/1232/094/rings.v3.skycell.1232.094.stk.g.unconv.fits&x=0.000000&y=0.000000&size=240&wcs=1&asinh=True&autoscale=99.500000
(22, 47)


In [18]:
image2diff = {}
image2median = {}
for x in diffImages:
    results = findName(x)
    image2diff[results[0]] = x
    image2median[results[0]] = results[1]
    
diff = [image2diff[x] for x in imname]
median = [image2median[x] for x in imname]

unmatchedObjects['diffImage'] = diff
unmatchedObjects['medianImage'] = median

In [19]:
# answer = [{'index': 45, 'timediff': 7.854872060001071, 'image': 'kplr2012242195726_ffi-cal.fits'},
#           {'index': 34, 'timediff': 13.539371960003336, 'image': 'kplr2011271191331_ffi-cal.fits'},
#           {'index': 37, 'timediff': 2.427519040000334, 'image': 'kplr2012004204112_ffi-cal.fits'}]
# ds9Options = []
# ccd = [45,1,44]
# for x in range(len(answer)):
#     imname = answer[x]['image']+'[{}]'.format(ccd[x])
#     subprocess.call(['ds9',imageDir+imname,'-regions','supernovae{}.reg'.format(x),'-scale','mode','zscale',
#                     diffDir+image2diff[imname],'-regions','supernovae{}.reg'.format(x),'-scale','mode','zscale',
#                     medianDir+image2median[imname],'-scale','mode','zscale','-frame','lock','image'])

In [76]:
uniqueName = np.unique(imname)
print(len(uniqueName))

2166


In [77]:
matches = []
for x in range(len(uniqueName)):
    name = uniqueName[x]
    print(name,x)
    diffname = image2diff[name]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['IMAGE_NAME'] == name]
    minitab = Table.from_pandas(mindf)
    try:
        matchtab = ascii.read('/home/mj1e16/outTablesImproved/'+name+'.csv')
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()

    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
    onematchTab = Table.from_pandas(onematch)
    if len(onematch) != 0:
        if len(matches) == 0:
            onematchTab = Table.from_pandas(onematch)
            matches = onematchTab

        else: 
            #onematchTab = Table.from_pandas(onematch)
            #print(onematchTab)
            for x in range(len(onematchTab)):
                matches.add_row(onematchTab[x])
                #print(onematchTab)
                #matches  = vstack(matches,onematchTab)
                #print(matches)
    

('kplr2009114174833_ffi-cal.fits[10]', 0)
('kplr2009114174833_ffi-cal.fits[13]', 1)
('kplr2009114174833_ffi-cal.fits[14]', 2)
('kplr2009114174833_ffi-cal.fits[16]', 3)
('kplr2009114174833_ffi-cal.fits[17]', 4)
('kplr2009114174833_ffi-cal.fits[19]', 5)
('kplr2009114174833_ffi-cal.fits[1]', 6)
('kplr2009114174833_ffi-cal.fits[21]', 7)
('kplr2009114174833_ffi-cal.fits[22]', 8)
('kplr2009114174833_ffi-cal.fits[23]', 9)
('kplr2009114174833_ffi-cal.fits[24]', 10)
('kplr2009114174833_ffi-cal.fits[26]', 11)
('kplr2009114174833_ffi-cal.fits[28]', 12)
('kplr2009114174833_ffi-cal.fits[2]', 13)
('kplr2009114174833_ffi-cal.fits[33]', 14)
('kplr2009114174833_ffi-cal.fits[34]', 15)
('kplr2009114174833_ffi-cal.fits[35]', 16)
('kplr2009114174833_ffi-cal.fits[36]', 17)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in log10


('kplr2009114174833_ffi-cal.fits[37]', 18)
('kplr2009114174833_ffi-cal.fits[39]', 19)
('kplr2009114174833_ffi-cal.fits[3]', 20)
('kplr2009114174833_ffi-cal.fits[42]', 21)
('kplr2009114174833_ffi-cal.fits[43]', 22)
('kplr2009114174833_ffi-cal.fits[46]', 23)
('kplr2009114174833_ffi-cal.fits[48]', 24)
('kplr2009114174833_ffi-cal.fits[4]', 25)
('kplr2009114174833_ffi-cal.fits[50]', 26)
('kplr2009114174833_ffi-cal.fits[51]', 27)
('kplr2009114174833_ffi-cal.fits[52]', 28)
('kplr2009114174833_ffi-cal.fits[53]', 29)
('kplr2009114174833_ffi-cal.fits[54]', 30)
('kplr2009114174833_ffi-cal.fits[55]', 31)
('kplr2009114174833_ffi-cal.fits[56]', 32)
('kplr2009114174833_ffi-cal.fits[57]', 33)
('kplr2009114174833_ffi-cal.fits[59]', 34)
('kplr2009114174833_ffi-cal.fits[5]', 35)
('kplr2009114174833_ffi-cal.fits[60]', 36)
('kplr2009114174833_ffi-cal.fits[61]', 37)
('kplr2009114174833_ffi-cal.fits[62]', 38)
('kplr2009114174833_ffi-cal.fits[63]', 39)
('kplr2009114174833_ffi-cal.fits[64]', 40)
('kplr20091141

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('kplr2009115053616_ffi-cal.fits[17]', 150)
('kplr2009115053616_ffi-cal.fits[21]', 151)
('kplr2009115053616_ffi-cal.fits[22]', 152)
('kplr2009115053616_ffi-cal.fits[24]', 153)
('kplr2009115053616_ffi-cal.fits[25]', 154)
('kplr2009115053616_ffi-cal.fits[26]', 155)
('kplr2009115053616_ffi-cal.fits[32]', 156)
('kplr2009115053616_ffi-cal.fits[33]', 157)
('kplr2009115053616_ffi-cal.fits[35]', 158)
('kplr2009115053616_ffi-cal.fits[36]', 159)
('kplr2009115053616_ffi-cal.fits[38]', 160)
('kplr2009115053616_ffi-cal.fits[39]', 161)
('kplr2009115053616_ffi-cal.fits[41]', 162)
('kplr2009115053616_ffi-cal.fits[45]', 163)
('kplr2009115053616_ffi-cal.fits[47]', 164)
('kplr2009115053616_ffi-cal.fits[48]', 165)
('kplr2009115053616_ffi-cal.fits[4]', 166)
('kplr2009115053616_ffi-cal.fits[50]', 167)
('kplr2009115053616_ffi-cal.fits[51]', 168)
('kplr2009115053616_ffi-cal.fits[54]', 169)
('kplr2009115053616_ffi-cal.fits[55]', 170)
('kplr2009115053616_ffi-cal.fits[56]', 171)
('kplr2009115053616_ffi-cal.fits[

...

('kplr2011177110110_ffi-cal.fits[33]', 1266)
('kplr2011177110110_ffi-cal.fits[34]', 1267)
('kplr2011177110110_ffi-cal.fits[35]', 1268)
('kplr2011177110110_ffi-cal.fits[42]', 1269)
('kplr2011177110110_ffi-cal.fits[44]', 1270)
('kplr2011177110110_ffi-cal.fits[46]', 1271)
('kplr2011177110110_ffi-cal.fits[47]', 1272)
('kplr2011177110110_ffi-cal.fits[4]', 1273)
('kplr2011177110110_ffi-cal.fits[52]', 1274)
('kplr2011177110110_ffi-cal.fits[53]', 1275)
('kplr2011177110110_ffi-cal.fits[54]', 1276)
('kplr2011177110110_ffi-cal.fits[57]', 1277)
('kplr2011177110110_ffi-cal.fits[58]', 1278)
('kplr2011177110110_ffi-cal.fits[60]', 1279)
('kplr2011177110110_ffi-cal.fits[62]', 1280)
('kplr2011177110110_ffi-cal.fits[63]', 1281)
('kplr2011177110110_ffi-cal.fits[64]', 1282)
('kplr2011177110110_ffi-cal.fits[65]', 1283)
('kplr2011177110110_ffi-cal.fits[66]', 1284)
('kplr2011177110110_ffi-cal.fits[67]', 1285)
('kplr2011177110110_ffi-cal.fits[68]', 1286)
('kplr2011177110110_ffi-cal.fits[69]', 1287)
('kplr20111

('kplr2011303191211_ffi-cal.fits[78]', 1449)
('kplr2011303191211_ffi-cal.fits[81]', 1450)
('kplr2011303191211_ffi-cal.fits[84]', 1451)
('kplr2011303191211_ffi-cal.fits[9]', 1452)
('kplr2011334181008_ffi-cal.fits[15]', 1453)
('kplr2011334181008_ffi-cal.fits[16]', 1454)
('kplr2011334181008_ffi-cal.fits[17]', 1455)
('kplr2011334181008_ffi-cal.fits[18]', 1456)
('kplr2011334181008_ffi-cal.fits[1]', 1457)
('kplr2011334181008_ffi-cal.fits[20]', 1458)
('kplr2011334181008_ffi-cal.fits[24]', 1459)
('kplr2011334181008_ffi-cal.fits[25]', 1460)
('kplr2011334181008_ffi-cal.fits[27]', 1461)
('kplr2011334181008_ffi-cal.fits[29]', 1462)
('kplr2011334181008_ffi-cal.fits[34]', 1463)
('kplr2011334181008_ffi-cal.fits[36]', 1464)
('kplr2011334181008_ffi-cal.fits[37]', 1465)
('kplr2011334181008_ffi-cal.fits[38]', 1466)
('kplr2011334181008_ffi-cal.fits[3]', 1467)
('kplr2011334181008_ffi-cal.fits[40]', 1468)
('kplr2011334181008_ffi-cal.fits[43]', 1469)
('kplr2011334181008_ffi-cal.fits[44]', 1470)
('kplr2011334

('kplr2012088132324_ffi-cal.fits[19]', 1632)
('kplr2012088132324_ffi-cal.fits[21]', 1633)
('kplr2012088132324_ffi-cal.fits[22]', 1634)
('kplr2012088132324_ffi-cal.fits[25]', 1635)
('kplr2012088132324_ffi-cal.fits[26]', 1636)
('kplr2012088132324_ffi-cal.fits[27]', 1637)
('kplr2012088132324_ffi-cal.fits[28]', 1638)
('kplr2012088132324_ffi-cal.fits[2]', 1639)
('kplr2012088132324_ffi-cal.fits[31]', 1640)
('kplr2012088132324_ffi-cal.fits[32]', 1641)
('kplr2012088132324_ffi-cal.fits[33]', 1642)
('kplr2012088132324_ffi-cal.fits[35]', 1643)
('kplr2012088132324_ffi-cal.fits[39]', 1644)
('kplr2012088132324_ffi-cal.fits[40]', 1645)
('kplr2012088132324_ffi-cal.fits[44]', 1646)
('kplr2012088132324_ffi-cal.fits[46]', 1647)
('kplr2012088132324_ffi-cal.fits[49]', 1648)
('kplr2012088132324_ffi-cal.fits[4]', 1649)
('kplr2012088132324_ffi-cal.fits[50]', 1650)
('kplr2012088132324_ffi-cal.fits[55]', 1651)
('kplr2012088132324_ffi-cal.fits[57]', 1652)
('kplr2012088132324_ffi-cal.fits[59]', 1653)
('kplr201208

('kplr2012211123923_ffi-cal.fits[64]', 1815)
('kplr2012211123923_ffi-cal.fits[65]', 1816)
('kplr2012211123923_ffi-cal.fits[69]', 1817)
('kplr2012211123923_ffi-cal.fits[71]', 1818)
('kplr2012211123923_ffi-cal.fits[72]', 1819)
('kplr2012211123923_ffi-cal.fits[73]', 1820)
('kplr2012211123923_ffi-cal.fits[74]', 1821)
('kplr2012211123923_ffi-cal.fits[78]', 1822)
('kplr2012211123923_ffi-cal.fits[79]', 1823)
('kplr2012211123923_ffi-cal.fits[82]', 1824)
('kplr2012242195726_ffi-cal.fits[10]', 1825)
('kplr2012242195726_ffi-cal.fits[11]', 1826)
('kplr2012242195726_ffi-cal.fits[16]', 1827)
('kplr2012242195726_ffi-cal.fits[17]', 1828)
('kplr2012242195726_ffi-cal.fits[19]', 1829)
('kplr2012242195726_ffi-cal.fits[21]', 1830)
('kplr2012242195726_ffi-cal.fits[23]', 1831)
('kplr2012242195726_ffi-cal.fits[24]', 1832)
('kplr2012242195726_ffi-cal.fits[25]', 1833)
('kplr2012242195726_ffi-cal.fits[28]', 1834)
('kplr2012242195726_ffi-cal.fits[2]', 1835)
('kplr2012242195726_ffi-cal.fits[35]', 1836)
('kplr20122

('kplr2013011160902_ffi-cal.fits[32]', 1998)
('kplr2013011160902_ffi-cal.fits[33]', 1999)
('kplr2013011160902_ffi-cal.fits[34]', 2000)
('kplr2013011160902_ffi-cal.fits[35]', 2001)
('kplr2013011160902_ffi-cal.fits[36]', 2002)
('kplr2013011160902_ffi-cal.fits[38]', 2003)
('kplr2013011160902_ffi-cal.fits[39]', 2004)
('kplr2013011160902_ffi-cal.fits[40]', 2005)
('kplr2013011160902_ffi-cal.fits[44]', 2006)
('kplr2013011160902_ffi-cal.fits[47]', 2007)
('kplr2013011160902_ffi-cal.fits[48]', 2008)
('kplr2013011160902_ffi-cal.fits[49]', 2009)
('kplr2013011160902_ffi-cal.fits[51]', 2010)
('kplr2013011160902_ffi-cal.fits[54]', 2011)
('kplr2013011160902_ffi-cal.fits[55]', 2012)
('kplr2013011160902_ffi-cal.fits[58]', 2013)
('kplr2013011160902_ffi-cal.fits[62]', 2014)
('kplr2013011160902_ffi-cal.fits[63]', 2015)
('kplr2013011160902_ffi-cal.fits[64]', 2016)
('kplr2013011160902_ffi-cal.fits[65]', 2017)
('kplr2013011160902_ffi-cal.fits[66]', 2018)
('kplr2013011160902_ffi-cal.fits[69]', 2019)
('kplr2013

In [79]:
bigNames = matches['IMAGE_NAME']
uniqueName = np.unique(bigNames)
magdiff = matches['magDiff']
magdiffsort = sorted(magdiff)
uniqueNameSort = [x for _,x in sorted(zip(magdiffsort,uniqueName))]
print(len(uniqueName))

14


In [80]:
#ascii.write(matches,'/home/mj1e16/outTablesImproved/firsthalfMatch.csv',format='csv',fast_writer=False)
ascii.write(matches,'/home/mj1e16/outTablesImproved/secondhalfMatch.csv',format='csv',fast_writer=False)

In [194]:
for x in range(len(matches)):
    openFromTable(matches,x)

[]
WARNING - panstarrs image note found ra = 807.239, dec = 369.9034
(807.239, 369.9034, 'kplr2009115053616_ffi-cal.fits[15]')
[]
WARNING - panstarrs image note found ra = 963.9432, dec = 798.7481
(963.9432, 798.7481, 'kplr2010049182302_ffi-cal.fits[51]')


KeyboardInterrupt: 

In [64]:
subprocess.call(['ds9', '/data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[57]', '-regions', '/home/mj1e16/outTablesImproved/testreg.reg', '-scale', 'mode', 'zscale', '/data/mj1e16/kepler/properDiff/diff_imNo6_channel25_ccd57.fits', '-regions', '/home/mj1e16/outTablesImproved/testreg.reg', '-scale', 'mode', 'zscale', '/data/mj1e16/kepler/medians/meidian_25_57.fits', '-regions', '/home/mj1e16/outTablesImproved/testreg.reg', '-scale', 'mode', 'zscale', '-frame', 'lock', 'image'])

0

In [153]:
print(magdiffsort[0])
print(uniqueNameSort[0])

-11.09146127898388
kplr2009114174833_ffi-cal.fits[15]


In [202]:
# goods = [2,10,14,20,32,37,41,44,47,77,84,94,97]
onematch

,ind,NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY,RA,DEC,NAME,index,diffImage,medianImage,magDiff,MAG_BEST_ORIG
210655,423122,4,18.2466,342.1980,118.2266,25.9895,1.137,0.120,288.753435,48.463141,kplr2009115002613_ffi-cal.fits[40],423122,diff_imNo37_channel24_ccd40.fits,meidian_24_40.fits,-4.749004,17.3825
210688,423186,262,16.0895,672.9335,541.5507,561.2244,1.284,0.221,288.675277,47.711241,kplr2009115002613_ffi-cal.fits[40],423186,diff_imNo37_channel24_ccd40.fits,meidian_24_40.fits,-4.260063,15.9854


In [203]:
goods = [41,94,97]
for x in goods:
    name = uniqueNameSort[x]
    diffname = image2diff[name]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['NAME'] == name]
    minitab = Table.from_pandas(mindf)
    try:
        matchtab = ascii.read('/data/mj1e16/kepler/sexKeplerCal/'+name+'.csv')   
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()
    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
    showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')
    print(x)

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in log10
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


41


IOError: [Errno 2] No such file or directory: '/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/kplr2009115131122_ffi-cal.fits[50].csv'

In [180]:
onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in log10
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['ds9', '/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits[2]', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/properDiff/diff_imNo11_channel30_ccd2.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/medians/meidian_30_2.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '-frame', 'lock', 'image']


In [198]:
matches = []
for x in range(len(uniqueName)):
    name = uniqueName[x]
    print(name,x)
    diffname = image2diff[name]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['NAME'] == name]
    minitab = Table.from_pandas(mindf)
    try:
        matchtab = ascii.read('/data/mj1e16/kepler/sexKeplerCal/'+name+'.csv')
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()

    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
    onematchTab = Table.from_pandas(onematch)
    if len(onematch) != 0:
        if len(matches) == 0:
            onematchTab = Table.from_pandas(onematch)
            matches = onematchTab

        else: 
            #onematchTab = Table.from_pandas(onematch)
            #print(onematchTab)
            for x in range(len(onematchTab)):
                matches.add_row(onematchTab[x])
                #print(onematchTab)
                #matches  = vstack(matches,onematchTab)
                #print(matches)
    

('kplr2009114174833_ffi-cal.fits[15]', 0)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in log10
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('kplr2009114174833_ffi-cal.fits[16]', 1)
('kplr2009114174833_ffi-cal.fits[17]', 2)
('kplr2009114174833_ffi-cal.fits[1]', 3)
('kplr2009114174833_ffi-cal.fits[21]', 4)
('kplr2009114174833_ffi-cal.fits[23]', 5)
('kplr2009114174833_ffi-cal.fits[28]', 6)
('kplr2009114174833_ffi-cal.fits[2]', 7)
('kplr2009114174833_ffi-cal.fits[33]', 8)
('kplr2009114174833_ffi-cal.fits[34]', 9)
('kplr2009114174833_ffi-cal.fits[35]', 10)
('kplr2009114174833_ffi-cal.fits[36]', 11)
('kplr2009114174833_ffi-cal.fits[38]', 12)
('kplr2009114174833_ffi-cal.fits[53]', 13)
('kplr2009114174833_ffi-cal.fits[54]', 14)
('kplr2009114174833_ffi-cal.fits[55]', 15)
('kplr2009114174833_ffi-cal.fits[56]', 16)
('kplr2009114174833_ffi-cal.fits[57]', 17)
('kplr2009114174833_ffi-cal.fits[58]', 18)
('kplr2009114174833_ffi-cal.fits[59]', 19)
('kplr2009114174833_ffi-cal.fits[5]', 20)
('kplr2009114174833_ffi-cal.fits[62]', 21)
('kplr2009114174833_ffi-cal.fits[67]', 22)
('kplr2009114174833_ffi-cal.fits[76]', 23)
('kplr2009114174833_ffi

In [199]:
ascii.write(matches,'/home/mj1e16/outTables15/matchplus3Magdiff.csv',format='csv',fast_writer=False)

In [102]:
showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')

['ds9', '/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits[10]', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/properDiff/diff_imNo11_channel70_ccd10.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/medians/meidian_70_10.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '-frame', 'lock', 'image']


In [159]:
matches = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)

[-0.05620325995845364, nan, nan, -0.7271288192655224, -0.014472199334716142, -0.05300994831146413, -1.4571815313378544, nan, -0.02180722264547895, -0.011935830200145239, -0.017960166210759618, -0.14517939577093023, -0.022554623422719782, -0.009808813321782576, -1.6606910087271756]


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in log10


In [16]:
ascii.write(unmatchedObjects,'/home/mj1e16/outTables15/transientsWnames.csv',format='csv',fast_writer=False)